Data cleanning: remove duplicate images and resize image to 256,256 and RGB mode

In [4]:
from data_preprocessing.data_cleanning import Data_cleanning
data_cleanning = Data_cleanning()
data_cleanning.remove_duplicated_image('ai_generated_images/')
data_cleanning.remove_duplicated_image('human_generated_images/')


ModuleNotFoundError: No module named 'imagehash'

Data Redcution: reduce the number of AI-generated images and human-genereated image to a balance state. due to space contrain on github, we gonna reduce the number of image to 50 using random sampling.

In [5]:
from data_preprocessing.data_reduction import Data_reduction
# if output "The number of images is less than the sampling size" that mean the dataset is already being reduced
Data_reduction = Data_reduction(50)
Data_reduction.random_sampling('ai_generated_images/')
Data_reduction.random_sampling('human_generated_images/')

50
The number of images is less than the sampling size
50
The number of images is less than the sampling size


Data labeling: lable the dataset for our supervised model

In [6]:
from data_preprocessing.data_labeling import Data_labeling
ai_data_labeling = Data_labeling()
#label ai-generated images as 0
ai_images, ai_label = ai_data_labeling.labeling('ai_generated_images/', 0)
human_data_labeling = Data_labeling()
#label human-generated images as 1
human_images, human_label = human_data_labeling.labeling('human_generated_images/', 1)
# For testing purpose we not going to save the data, using the exsiting npy file for train the model.
# np.save('ai_images.npy', ai_images)
# np.save('ai_label.npy', ai_label)
# np.save('human_images.npy', human_images)
# np.save('human_label.npy', human_label)
print('ai-generated image array shape: ',ai_images.shape) #(2999, 256, 256, 3)
print('ai-generated label array shape: ',ai_label.shape) #(2999,)
print('human-generated image array shape: ',human_images.shape)#(2999, 256, 256, 3)
print('human-generated label array shape: ',human_label.shape)#(2999,)

ai-generated image array shape:  (50, 256, 256, 3)
ai-generated label array shape:  (50,)
human-generated image array shape:  (50, 256, 256, 3)
human-generated label array shape:  (50,)


In [44]:
from model_training.convultional_neural_net import CNN
import numpy as np
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

#print('This is the ai_image: ', ai_images)
#print('This is the labels: ', ai_label)
#ai_data = np.concatenate(ai_label, ai_images)
#human_data = np.concatenate(human_label, human_images)
labels = ai_label + human_label
data = ai_images + human_images
#training_data = np.append(ai_data, human_data)

X = data
y = labels

# Split the dataset into training and testing sets
print('It got here and this is the ai_images: ', ai_images.shape)
print('It got here and this is the x size: ', X.shape)
print('It got here and this is the y size: ', y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)


# Create and train the SVM model
svm_model = SVC(kernel='linear')  # You can choose other kernels like 'rbf', 'poly', etc.
svm_model.fit(X_train, y_train)

# Make predictions
y_pred = svm_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{report}")

It got here and this is the ai_images:  (50, 256, 256, 3)
It got here and this is the x size:  (50, 256, 256, 3)
It got here and this is the y size:  (50,)


ValueError: Found array with dim 4. SVC expected <= 2.

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

train_images, test_images = X_train / 255.0, X_test / 255.0

# Add a channel dimension (required by the Conv2D layer)
train_images = train_images.reshape((train_images.shape[0], 28, 28, 1))
test_images = test_images.reshape((test_images.shape[0], 28, 28, 1))

# Build the CNN model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

# Add Dense layers on top
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(train_images, y_train, epochs=5, 
                    validation_data=(test_images, y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, y_test)
print(f"Test accuracy: {test_acc}")

# Plot training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')
plt.show()